# ImageToolkit Notebook — Roll 22671A7331

This notebook accompanies the Streamlit app and the detailed PDF report. It contains runnable code and explanations for each processing step.

---

In [ ]:
# Setup: imports. Run this cell to prepare.
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# helper to show images inline
def show(img, title=None):
    if img is None:
        print('None image')
        return
    if isinstance(img, np.ndarray):
        if img.ndim == 2:
            plt.imshow(img, cmap='gray')
        else:
            # assume BGR (OpenCV) -> convert to RGB for display
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()


## Example Image
We'll create a synthetic image for experiments (so the notebook runs without external files).

In [ ]:
# Create a synthetic test image (512x512) with gradients and shapes
def make_synthetic_image(w=512, h=512):
    x = np.linspace(0, 255, w, dtype=np.uint8)
    y = np.linspace(0, 255, h, dtype=np.uint8)
    xv, yv = np.meshgrid(x, y)
    b = xv
    g = yv
    r = ((xv.astype(np.uint16) + yv.astype(np.uint16)) // 2).astype(np.uint8)
    img = np.stack([b, g, r], axis=2)
    cv2.circle(img, (w//2, h//2), min(w,h)//6, (255,255,255), thickness=-1)
    cv2.rectangle(img, (10,10), (w-10, h//4), (0,0,0), thickness=-1)
    return img

img = make_synthetic_image(512, 512)
show(img, 'Synthetic Test Image')


## Color Space Conversions
Convert between BGR, Grayscale, HSV, and YCrCb.

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
show(gray, 'Grayscale')

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
show(hsv, 'HSV (displayed as RGB - not perceptual)')

ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
show(ycrcb, 'YCrCb (displayed as RGB - not perceptual)')


## Filtering & Morphology
Gaussian, Median, Mean filtering and morphological operations.

In [ ]:
gblur = cv2.GaussianBlur(img, (7,7), 0)
show(gblur, 'Gaussian Blur (7x7)')

median = cv2.medianBlur(img, 7)
show(median, 'Median (7)')

mean = cv2.blur(img, (7,7))
show(mean, 'Mean filter (7x7)')

# Morphology on grayscale edges
edges = cv2.Canny(gray, 50, 150)
show(edges, 'Canny edges')

kernel = np.ones((5,5), np.uint8)
dilated = cv2.dilate(edges, kernel, iterations=1)
show(dilated, 'Dilated edges')

eroded = cv2.erode(edges, kernel, iterations=1)
show(eroded, 'Eroded edges')


## Enhancement
Histogram equalization and contrast stretching.

In [ ]:
heq = cv2.equalizeHist(gray)
show(heq, 'Histogram Equalized (grayscale)')

# Contrast stretching example (linear between percentiles)
p2, p98 = np.percentile(img, (2, 98))
stretched = np.clip((img - p2) * 255.0 / (p98 - p2), 0, 255).astype(np.uint8)
show(stretched, 'Contrast Stretched')


## Edge Detection
Sobel, Laplacian, and Canny examples.

In [ ]:
sobx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
soby = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
mag = np.sqrt(sobx**2 + soby**2)
mag = np.uint8(np.clip(mag / mag.max() * 255, 0, 255))
show(mag, 'Sobel Magnitude')

lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.clip(np.abs(lap), 0, 255))
show(lap, 'Laplacian')

canny = cv2.Canny(gray, 50, 150)
show(canny, 'Canny (50,150)')


## Compression & Saving
Save outputs in different formats and compare file sizes.

In [ ]:
out_dir = '/mnt/data'
import os
os.makedirs(out_dir, exist_ok=True)

def save_and_report(img, name, fmt='PNG', quality=90):
    fname = os.path.join(out_dir, f'notebook_22671A7331_' + name + '.' + fmt.lower())
    # convert to PIL for saving convenience
    if isinstance(img, np.ndarray):
        if img.ndim == 3:
            pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            pil = Image.fromarray(img)
    else:
        pil = img
    if fmt.lower() in ('jpg','jpeg'):
        pil.save(fname, format='JPEG', quality=quality)
    else:
        pil.save(fname, format=fmt.upper())
    size = os.path.getsize(fname)
    print(f'Saved {fname} -> {size} bytes')
    return fname, size

save_and_report(img, 'original', fmt='PNG')
save_and_report(img, 'original_jpeg', fmt='JPEG', quality=85)
save_and_report(img, 'original_bmp', fmt='BMP')


## Conclusion
This notebook and the Streamlit app illustrate core techniques in image processing. Modify parameters and try with real images to learn more.